**FINDING F1-SCORE FOR CHATGPT RESPONSE AND GROUND TRUTH RESPONSES**

In [1]:
from collections import defaultdict

def calculate_scores(output_file, ground_truth_file, classes):
    with open(output_file, 'r') as f:
        output_data = f.read().strip().split('\n')

    with open(ground_truth_file, 'r') as f:
        ground_truth_data = f.read().strip().split('\n')

    tp = defaultdict(int)
    fp = defaultdict(int)
    fn = defaultdict(int)
    correct = 0
    total = 0

    for output_line, ground_truth_line in zip(output_data, ground_truth_data):
        output_tokens = output_line.strip().split()
        ground_truth_tokens = ground_truth_line.strip().split()

        for output_token, ground_truth_token in zip(output_tokens, ground_truth_tokens):
            if output_token == ground_truth_token:
                correct += 1
                tp[output_token] += 1
            else:
                fp[output_token] += 1
                fn[ground_truth_token] += 1
            total += 1

    precision = {}
    recall = {}
    f1_score = {}
    accuracy = correct / total if total > 0 else 0
    macro_f1_score = 0.0

    for class_label in classes:
        precision[class_label] = tp[class_label] / (tp[class_label] + fp[class_label]) if (tp[class_label] + fp[class_label]) > 0 else 0
        recall[class_label] = tp[class_label] / (tp[class_label] + fn[class_label]) if (tp[class_label] + fn[class_label]) > 0 else 0
        f1_score[class_label] = 2 * (precision[class_label] * recall[class_label]) / (precision[class_label] + recall[class_label]) if (precision[class_label] + recall[class_label]) > 0 else 0
        macro_f1_score += f1_score[class_label]

    macro_f1_score /= len(classes)

    return precision, recall, f1_score, macro_f1_score, accuracy

classes = ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']

precision, recall, f1_score, macro_f1_score, accuracy = calculate_scores('/kaggle/input/abcdef/ChatGPT Answers.txt', '/kaggle/input/abcdef/ground_truth.txt', classes)

for class_label in classes:
    print(f"Class: {class_label}")
    print(f"Precision: {precision[class_label]}")
    print(f"Recall: {recall[class_label]}")
    print(f"F1-score: {f1_score[class_label]}")
    print()

print(f"Macro F1-score: {macro_f1_score}")
print(f"Accuracy: {accuracy}")

Class: O
Precision: 0.9518716577540107
Recall: 0.9709090909090909
F1-score: 0.9612961296129614

Class: B-PER
Precision: 0.6923076923076923
Recall: 0.8181818181818182
F1-score: 0.7500000000000001

Class: I-PER
Precision: 0.6363636363636364
Recall: 0.5
F1-score: 0.56

Class: B-LOC
Precision: 0.8
Recall: 0.8
F1-score: 0.8000000000000002

Class: I-LOC
Precision: 0.5
Recall: 1.0
F1-score: 0.6666666666666666

Class: B-ORG
Precision: 0.7142857142857143
Recall: 1.0
F1-score: 0.8333333333333333

Class: I-ORG
Precision: 0.75
Recall: 1.0
F1-score: 0.8571428571428571

Class: B-MISC
Precision: 0.8
Recall: 0.32
F1-score: 0.45714285714285713

Class: I-MISC
Precision: 0.6
Recall: 0.5
F1-score: 0.5454545454545454

Macro F1-score: 0.7145595988170245
Accuracy: 0.9252782193958664


**FINDING F1-SCORE FOR IndicBERT AND GROUND TRUTH RESPONSES**

In [8]:
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = []
        for line in lines:
            line = eval(line.strip())
            tags = [tag[1] for tag in line]
            data.append(tags)
        return data

def calculate_metrics(ground_truth, output):
    metrics = defaultdict(lambda: defaultdict(float))
    all_ground_truth = [tag for tags in ground_truth for tag in tags]
    all_output = [tag for tags in output for tag in tags]
    
    accuracy = accuracy_score(all_ground_truth, all_output)

    for label in ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']:
        y_true = [tag == label for tags in ground_truth for tag in tags]
        y_pred = [tag == label for tags in output for tag in tags]
        precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
        metrics[label]['precision'] = precision
        metrics[label]['recall'] = recall
        metrics[label]['f1_score'] = f1_score
    return metrics, accuracy

def print_metrics(metrics, accuracy):
    for label, values in metrics.items():
        print(f"Metrics for {label}:")
        print(f"Precision: {values['precision']:.4f}")
        print(f"Recall: {values['recall']:.4f}")
        print(f"F1 Score: {values['f1_score']:.4f}")
        print()
    print(f"Accuracy: {accuracy:.4f}")

def calculate_macro_f1(metrics):
    f1_scores = [values['f1_score'] for values in metrics.values()]
    macro_f1 = sum(f1_scores) / len(f1_scores)
    return macro_f1

ground_truth_file = '/kaggle/input/indicbert-data/indicbert_gt.txt'
output_file = '/kaggle/input/indicbert-data/indicbert_prediction.txt'

ground_truth_data = read_file(ground_truth_file)
output_data = read_file(output_file)

metrics, accuracy = calculate_metrics(ground_truth_data, output_data)

print_metrics(metrics, accuracy)

macro_f1 = calculate_macro_f1(metrics)
print(f"Macro F1 Score: {macro_f1:.4f}")

Metrics for O:
Precision: 0.9428
Recall: 0.9865
F1 Score: 0.9642

Metrics for B-PER:
Precision: 0.4615
Recall: 0.5455
F1 Score: 0.5000

Metrics for I-PER:
Precision: 0.8889
Recall: 0.5714
F1 Score: 0.6957

Metrics for B-LOC:
Precision: 0.8750
Recall: 0.7000
F1 Score: 0.7778

Metrics for I-LOC:
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for B-ORG:
Precision: 0.6667
Recall: 0.8000
F1 Score: 0.7273

Metrics for I-ORG:
Precision: 0.8000
Recall: 0.8000
F1 Score: 0.8000

Metrics for B-MISC:
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for I-MISC:
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Accuracy: 0.9247
Macro F1 Score: 0.4961


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**FINDING F1-SCORE FOR IndicNER RESPONSE AND GROUND TRUTH RESPONSES**

In [10]:
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = []
        for line in lines:
            line = eval(line.strip())
            tags = [tag[1] for tag in line]
            data.append(tags)
        return data

def calculate_metrics(ground_truth, output):
    metrics = defaultdict(lambda: defaultdict(float))
    all_ground_truth = [tag for tags in ground_truth for tag in tags]
    all_output = [tag for tags in output for tag in tags]
    
    accuracy = accuracy_score(all_ground_truth, all_output)

    for label in ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']:
        y_true = [tag == label for tags in ground_truth for tag in tags]
        y_pred = [tag == label for tags in output for tag in tags]
        precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
        metrics[label]['precision'] = precision
        metrics[label]['recall'] = recall
        metrics[label]['f1_score'] = f1_score
    return metrics, accuracy

def print_metrics(metrics, accuracy):
    for label, values in metrics.items():
        print(f"Metrics for {label}:")
        print(f"Precision: {values['precision']:.4f}")
        print(f"Recall: {values['recall']:.4f}")
        print(f"F1 Score: {values['f1_score']:.4f}")
        print()
    print(f"Accuracy: {accuracy:.4f}")

def calculate_macro_f1(metrics):
    f1_scores = [values['f1_score'] for values in metrics.values()]
    macro_f1 = sum(f1_scores) / len(f1_scores)
    return macro_f1

ground_truth_file = '/kaggle/input/indicner-dataset/IndicNER_gt.txt'
output_file = '/kaggle/input/indicner-dataset/IndicNER_prediction.txt'

ground_truth_data = read_file(ground_truth_file)
output_data = read_file(output_file)

metrics, accuracy = calculate_metrics(ground_truth_data, output_data)

print_metrics(metrics, accuracy)

macro_f1 = calculate_macro_f1(metrics)
print(f"Macro F1 Score: {macro_f1:.4f}")


Metrics for O:
Precision: 0.9359
Recall: 0.9827
F1 Score: 0.9587

Metrics for B-PER:
Precision: 0.6667
Recall: 0.5455
F1 Score: 0.6000

Metrics for I-PER:
Precision: 0.7500
Recall: 0.4286
F1 Score: 0.5455

Metrics for B-LOC:
Precision: 0.8889
Recall: 0.7273
F1 Score: 0.8000

Metrics for I-LOC:
Precision: 1.0000
Recall: 0.5000
F1 Score: 0.6667

Metrics for B-ORG:
Precision: 0.3333
Recall: 1.0000
F1 Score: 0.5000

Metrics for I-ORG:
Precision: 0.8333
Recall: 0.8333
F1 Score: 0.8333

Metrics for B-MISC:
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for I-MISC:
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Accuracy: 0.9184
Macro F1 Score: 0.5449


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
